# Universidade NOVA de Lisboa
## Demo 1: Interação com o CLI via Paramiko (Python 🐍)

É necessário efetuar o inventário das configurações das interfaces de uma rede de dados.

Obter informações de um dispositivo na rede é relativamente simples. Os dispositivos têm comandos diferentes na sua CLI para consultar configurações distintas. Por exemplo, o comando ```show ip int brief``` permite a leitura dos parâmetros de uma interface.

No entanto, o que acontece se é necessário obter apenas um destes parâmetros? E se tiver muitos dispositivos e interfaces?

O login via SSH, a cópia/colagem dos comandos e o processamento dos dados podem demorar muito tempo ⏳. Além disso, é muito provável que ocorram muitos erros no processo 🔥🔥🔥.


### O que faremos nesta demonstração

- Explorar a biblioteca Python paramiko para interagir com a CLI de um dispositivo Cisco via SSH.
- Estabelecer a conexão com o dispositivo e obter as configurações das interfaces.
- Filtrar a informação para obter parâmetros específicos.
- Configurar novos parâmetros no dispositivo.


Primeiro, vamos importar as seguintes bibliotecas

In [35]:
import paramiko
from paramiko_expect import SSHClientInteraction
import logging
import re

Neste laboratorio temos um dispositivo Cisco IOSXR. O paso seguinte é a especificação dos parâmetros para a conexão

In [36]:
CISCO_IP = "192.168.10.4"
CISCO_PORT = 22
CISCO_USERNAME = "vagrant"
CISCO_PASSWORD = "vagrant"

Agora, com a biblioteca ```paramiko``` vamos fazer a conexão via SSH. Esta biblioteca permite diferentes configurações para a conexão, por exemplo a administração de chaves, Proxy, SFTP, etc. Para a nossa demo, usaremos a seguinte configuração:

In [37]:
remote_conn_pre=paramiko.SSHClient()
remote_conn_pre.set_missing_host_key_policy(paramiko.AutoAddPolicy())
remote_conn_pre.connect(
    CISCO_IP, 
    port = CISCO_PORT,
    username = CISCO_USERNAME,  
    password = CISCO_PASSWORD,
    look_for_keys=False,
    allow_agent=False)


Normalmente, os comandos da CLI da Cisco demoram algum tempo a gerar os resultados. Quando interagimos manualmente via SSH, aguardamos até que o dispositivo entregue os resultados de um comando para introduzir o seguinte.

Num processo automatizado, muitas vezes é impossível saber quanto tempo isso demorará. Nesse caso, é possível utilizar a utilidade ```time.sleep``` para definir um limite de tempo, bem como os plugins disponíveis na biblioteca paramiko.

Nesta demonstração, utilizaremos o plugin ```SSHClientInteraction```. Este plugin aguarda até que um prompt conhecido seja recebido para continuar a execução do programa.

In [38]:
CISCO_PROMPT = "RP/0/RP0/CPU0:test_XR#"

remote_conn = SSHClientInteraction(remote_conn_pre, display=False)
print remote_conn

Finalmente, com a conexão pronta vamos obter as configurações das interfaces de rede do dispositivo

In [49]:
remote_conn.send("show ip int brief\n")
remote_conn.expect(CISCO_PROMPT)
output = remote_conn.current_output_clean
print(output)

show ip int brief

Sat Sep 16 19:58:38.989 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
GigabitEthernet0/0/0/0         192.168.10.4    Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.0.2.15       Up              Up       default 
RP/0/RP0/CPU0:test_XR#



Que informações correspondem realmente às interfaces do dispositivo? É aqui que é possível utilizar programação para o processamento dos dados. Neste caso, usaremos filtros regex de Python.

In [41]:
interface=re.search(r'(GigabitEthernet|FastEthernet|Serial|Tunnel|Loopback|Dialer|BVI|Vlan|Virtual-Access)\
[0-9]{1,4}/?[0-9]{0,4}.?[0-9]{0,4}/?[0-9]{0,3}/?[0-9]{0,3}/?[0-9]{0,3}:?[0-9]{0,3}',output)

if interface:
    interface = interface.group()
print "Encontrada a interface: "+interface

Encontrada a interface: GigabitEthernet0/0/0/0


Agora, tentaremos fazer novas configurações no dispositivo. Mas o que acontece quando as configurações não estão corretas?

O dispositivo notificará quais configurações estão incorretas. A biblioteca é apenas um meio de comunicação entre o script e o dispositivo e não efetua validações, controle de configurações parciais, etc.

In [51]:
remote_conn.send('''
conf t
router static
address-family ipv4 unicast 
1.1.1.0/24 192.168.10.5
1.1.2.0./24 192.168.10.5
commit
end
''')
remote_conn.expect(CISCO_PROMPT)
output = remote_conn.current_output_clean
print(output)


RP/0/RP0/CPU0:test_XR#conf t

Sat Sep 16 20:00:23.144 UTC
RP/0/RP0/CPU0:test_XR(config)#router static
RP/0/RP0/CPU0:test_XR(config-static)#address-family ipv4 unicast 
RP/0/RP0/CPU0:test_XR(config-static-afi)#1.1.1.0/24 192.168.10.5
RP/0/RP0/CPU0:test_XR(config-static-afi)#1.1.2.0./24 192.168.10.5
                                                ^
% Invalid input detected at '^' marker.
RP/0/RP0/CPU0:test_XR(config-static-afi)#commit
Sat Sep 16 20:00:23.830 UTC
RP/0/RP0/CPU0:test_XR(config-static-afi)#end
RP/0/RP0/CPU0:test_XR#



É possível validar que apenas uma das rotas estáticas foi configurada no dispositivo

In [53]:
remote_conn.send("show route static\n")
remote_conn.expect(CISCO_PROMPT)
output = remote_conn.current_output_clean
print(output)

show route static

Sat Sep 16 20:01:35.167 UTC

S*   0.0.0.0/0 [1/0] via 10.0.2.2, 4d03h, MgmtEth0/RP0/CPU0/0
S    1.1.1.0/24 [1/0] via 192.168.10.5, 00:06:58
RP/0/RP0/CPU0:test_XR#

